# GPS Fitness Analysis - From Garmin Fenix 5 and GPX Files

In [ ]:
import matplotlib as plt
import numpy as np
import pandas as pd
import gpxpy
import datetime
from geopy import distance
from math import sqrt, floor
import numpy as np
import pandas as pd
#import plotly.plotly as py
import plotly.graph_objs as go
import haversine

In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from collections import Counter, defaultdict
from datetime import datetime
import xml.etree.ElementTree as ET

## Importing files

In [23]:
with open("activity_5070523877.gpx") as fp:
    soup = BeautifulSoup(fp, "html.parser")

In [3]:
#soup

In [4]:
soup.trk.find_all("type")[0].get_text()

'hiking'

In [5]:
soup.trk.find_all("name")[0].get_text()

'Bear Peak and South Boulder Peak'

In [6]:
points = soup.find_all("trkpt")

In [7]:
len(points)

6406

## Unpacking Latitude and Longitude

In [8]:
dir(points[0])

['__bool__',
 '__call__',
 '__class__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_all_strings',
 '_find_all',
 '_find_one',
 '_is_xml',
 '_lastRecursiveChild',
 '_last_descendant',
 '_should_pretty_print',
 'append',
 'attrs',
 'can_be_empty_element',
 'cdata_list_attributes',
 'childGenerator',
 'children',
 'clear',
 'contents',
 'decode',
 'decode_contents',
 'decompose',
 'decomposed',
 'descendants',
 'encode',
 'encode_contents',
 'extend',
 'extract',
 'fetchNextSiblings',
 'fetchParents',
 'fetchPrevious',
 'fetchPreviousSiblings',
 '

In [9]:
points[0]

<trkpt lat="39.94084543548524379730224609375" lon="-105.26018234901130199432373046875">
<ele>1732.5999755859375</ele>
<time>2020-06-07T12:11:16.000Z</time>
<extensions>
<ns3:trackpointextension>
<ns3:atemp>23.0</ns3:atemp>
<ns3:hr>131</ns3:hr>
</ns3:trackpointextension>
</extensions>
</trkpt>

In [10]:
points[0].attrs

{'lat': '39.94084543548524379730224609375',
 'lon': '-105.26018234901130199432373046875'}

In [11]:
float(points[0].attrs['lat'])

39.940845435485244

## Unpacking Elevation, HR, and Ambient Temp

In [12]:
float(
    points[0].ele.get_text()
) * 3.28    #Elavation converted from meters to feet

5682.9279199218745

In [13]:
float(
    points[0].extensions.findChildren()[2].get_text()
) #HR

131.0

In [14]:
float(
    points[0].extensions.findChildren()[1].get_text()
) #Ambient Temp

23.0

## Unpacking Date Time

In [15]:
date_format = "%Y-%m-%dT%H:%M:%S.%fZ" 

datetime.strptime(
    points[0].time.get_text(),
    date_format
) #Datetime

datetime.datetime(2020, 6, 7, 12, 11, 16)

## Putting it all into a Pandas Dataframe

In [16]:
lat = []
lon = []
ele = []
time = []
hr = []
temp = []

date_format = "%Y-%m-%dT%H:%M:%S.%fZ" 

for p in points:
    lat.append(float(p.attrs['lat']))
    lon.append(float(p.attrs['lat']))
    ele.append(float(p.ele.get_text()) * 3.28) #3.28 is for converting feet to meters
    time.append(datetime.strptime(p.time.get_text(), date_format))
    hr.append(float([item.get_text() for item in p.findChildren()][5]))
    temp.append(float([item.get_text() for item in p.findChildren()][4]))

In [19]:
print(len(lat))
print(len(lon))
print(len(ele))
print(len(time))
print(len(hr))
print(len(temp))

6406
6406
6406
6406
6406
6406


In [20]:
time[1000] - time[0]

datetime.timedelta(seconds=2520)

In [21]:
boulder_df = []
boulder_df = pd.DataFrame(columns=['lon', 'lat', 'ele', 'time', 'hr', 'temp', 'time_elap', 'elev_gain', 'dist'])

boulder_df.lon = lon
boulder_df.lat = lat
boulder_df.ele = ele
boulder_df.time = time
boulder_df.hr = hr
boulder_df.temp = temp
boulder_df.time_elap = boulder_df.time - boulder_df.time[0]
#boulder_df.elev_gain = 
#boulder_df.dist = 

In [22]:
boulder_df

,lon,lat,ele,time,hr,temp,time_elap,elev_gain,dist
0,39.940845,39.940845,5682.92792,2020-06-07 12:11:16,131.0,23.0,00:00:00,NaN,NaN
1,39.940854,39.940854,5722.28792,2020-06-07 12:11:17,130.0,23.0,00:00:01,NaN,NaN
2,39.940865,39.940865,5740.65584,2020-06-07 12:11:18,131.0,23.0,00:00:02,NaN,NaN
3,39.940883,39.940883,5751.80792,2020-06-07 12:11:19,131.0,23.0,00:00:03,NaN,NaN
4,39.940905,39.940905,5756.40000,2020-06-07 12:11:20,131.0,22.0,00:00:04,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6401,39.938618,39.938618,5772.80000,2020-06-07 16:36:35,107.0,33.0,04:25:19,NaN,NaN
6402,39.938625,39.938625,5773.45584,2020-06-07 16:36:39,108.0,33.0,04:25:23,NaN,NaN
6403,39.938627,39.938627,5773.45584,2020-06-07 16:36:40,114.0,33.0,04:25:24,NaN,NaN
6404,39.938635,39.938635,5775.42416,2020-06-07 16:36:42,119.0,33.0,04:25:26,NaN,NaN
